In [1]:
import os
import boto3
import pandas as pd

In [2]:
# Initialize the S3 client
s3_client = boto3.client('s3')

# Bucket name and file paths
bucket_name = 'ca-climate-index'
directory = '0_map_data/crosswalk_data/CCI_Projects_Project_Category_Update_02142024.xlsm'

print('Pulling file')
s3_client.download_file(bucket_name, directory, 'CCI_Projects_Project_Category_Update_02142024.xlsm')
print('File pulled')

Pulling file
File pulled


In [3]:
crosswalk_data = pd.read_excel('CCI_Projects_Project_Category_Update_02142024.xlsm')

## How many rows within the original dataset?

In [18]:
pd.set_option('display.max_columns', None)
print('Number of rows within dataset:', len(crosswalk_data))
display(crosswalk_data)


Number of rows within dataset: 133698


,Project IDNumber,Reporting Cycle Name,Agency Name,Program Name,Program Description,Sub Program Name,Record Type,Project Name,Project Type,Project Description,SECTOR,CATEGORY,ACTION,Census Tract,Address,Lat Long,Senate\nDistrict,Assembly\nDistrict,County,Total Project Cost,Total Program GGRFFunding,Project Life Years,Total Project GHGReductions,Annual Project GHGReductions,Project Count,Fiscal Year Funding Project,Is Benefit Disadvantaged Communities,Disadvantaged Community Criteria,Disadvantaged Community Need,Disadvantaged Community Census Tracts,Total GGRFDisadvantaged Community Funding,Disadvantaged Community Benefits Description,Funding Benefiting Disadvantaged Communities,Estimated Num Vehicles In Service,Funding Within Disadvantage Communities,Other Project Benefits Description,VMTReductions,Number Of Housing Units,Number Of Affordable Housing Units,Estimated Number Of Trees To Be Planted,Energy Cost Savings,Estimated Energy Saved KWH,Estimated Energy Saved Therms,Estimated Water Saved Gallons,Estimated Energy Generated KWH,Estimated Fuel Use Reduction Gal,Vouchers Benefiting Disadvantaged Communities,Number Of Rebates Issued,Rebates Within Disadvantaged Communities,Date Operational,Project Completion Date,Date Imported,Funding Recipient,AB1550Choice,Buffer Amount,Buffer Count,Is AB1550Buffer Region,CESVersion,CESVersion Calc,DAC1550Amount,DAC1550Count,Is Benefit DAC1550Communities,DACTable,FG17Comm Need,FG17Comm Need Qual,DACCommunity Benefit Critieria Met,Date Selected For Award,Low Income Amount,Low Income Count,Low Income Household Amount,Low Income Household Count,Is Low Income Communities,Potential Buffer Amount,Potential Buffer Count,Potential DAC1550Amount,Potential DAC1550Count,Potential Low Income Amount,Potential Low Income Count,Estimated Acres Preserved,Estimated Acres Restored,Estimated Acres Treated,Estimated Diverted From Landfills Tons,Renewable Fuel Generation Tons,Wood Burning Reduction Cords,Estimated Ridership Increases,Black Carbon Reductions Pounds,StateEW_DPM,StateEW_Nox,StateEW_PM25,StateEW_ROG,Diesel Pm Reductions Pounds,NOx Reductions Pounds,Pm25Reductions Pounds,Rog Reductions Pounds,Estimated Total Recycling Tons,Estimated Waste Digested Tons,Reclaimed Food Tons,NUMBER OF PLANS,POLLINATOR ACREAGE,RESEARCH GRANT,SCIENCE ADVANCEMENT,SOIL BENEFIT,TRAVEL COST SAVINGS,EST ENERGY GEN SCF,EST SOURCE RED TONS,FUEL TREATMENT NUM,EDUCATION EVENT NUM,ENERGY AUDIT BUILDINGS,EST DIVERT LANDFILLS TONS,EST FUEL GEN GAL,Voucher ID,Voucher Name,Voucher Description,Direct Jobs Fte,Indirect Jobs Fte,Induced Jobs Fte,Climate Adaptation,Community Engagement,Compost Produced Tons,Compost Produced Tons Yr,Net Density DUA,Applicants Assisted,Invasive Cover 12 Months,Invasive Cover 36 Months,Project Acreage,IS IAE,Intermediary Admin Expenses Calc,PRIMARY_FUNDING_RECIPIENT_TYPE,TRIBAL AFFILIATION,PROJECT PARTNERS
0,G13-AQIP-01,2015,California Air Resources Board,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clean Vehicle Rebate Project,IMPLEMENT,NaN,NaN,CVRP promotes clean vehicle adoption in Califo...,"Zero-Emission Vehicles, Equipment, and Infrast...",Light-Duty Vehicles,Consumer Incentives,6.001400e+09,NaN,NaN,07,14,Alameda,14833.0,14833.0,3,50.0,0.0,7.0,NaN,0.0,NaN,NaN,NaN,0.0,Reduction of Criteria Air Pollutant or Toxic A...,0.0,0.0,0.0,CVRP promotes clean vehicle adoption in Califo...,0.0,0.0,0.0,0.0,9455.0,0.0,0.0,0.0,0.0,5110.0,0.0,7.0,0.0,2013-09-21,01/05/2015,2023-06-05 09:18:14.087,Center for Sustainable Energy (CSE),NaN,NaN,NaN,0.0,2.0,535.0,NaN,NaN,0.0,Sustainable Transportation,D,Common Need,A,NaT,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,CVRP,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,G13-AQIP-01,2015,California Air Resources Board,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clea

## Display all columns

In [19]:
print('Number of columns:', len(crosswalk_data.columns.tolist()))
display(crosswalk_data.columns.tolist())

Number of columns: 130


['Project IDNumber',
 'Reporting Cycle Name',
 'Agency Name',
 'Program Name',
 'Program Description',
 'Sub Program Name',
 'Record Type',
 'Project Name',
 'Project Type',
 'Project Description',
 'SECTOR',
 'CATEGORY',
 'ACTION',
 'Census Tract',
 'Address',
 'Lat Long',
 'Senate\nDistrict',
 'Assembly\nDistrict',
 'County',
 'Total Project Cost',
 'Total Program GGRFFunding',
 'Project Life Years',
 'Total Project GHGReductions',
 'Annual Project GHGReductions',
 'Project Count',
 'Fiscal Year Funding Project',
 'Is Benefit Disadvantaged Communities',
 'Disadvantaged Community Criteria',
 'Disadvantaged Community Need',
 'Disadvantaged Community Census Tracts',
 'Total GGRFDisadvantaged Community Funding',
 'Disadvantaged Community Benefits Description',
 'Funding Benefiting Disadvantaged Communities',
 'Estimated Num Vehicles In Service',
 'Funding Within Disadvantage Communities',
 'Other Project Benefits Description',
 'VMTReductions',
 'Number Of Housing Units',
 'Number Of Aff

## Selecting columns relevant to initial analysis

In [10]:
relevant_columns = [
    'Program Name',
    'Program Description',
    'Sub Program Name',
    'Project Type',
    'Project Description',
    'SECTOR',
    'CATEGORY',
    'ACTION',
    'Census Tract',
    'Total Project GHGReductions',
    'Project Count'
]

In [11]:
data_of_interest = crosswalk_data[relevant_columns]

In [ ]:
# Set display options to show all columns and rows
# pd.set_option('display.max_columns', None)  # To display all columns
# pd.set_option('display.max_rows', None)     # To display all rows

# Now display data_of_interest
display(data_of_interest)

,Program Name,Program Description,Sub Program Name,Project Type,Project Description,SECTOR,CATEGORY,ACTION,Census Tract,Total Project GHGReductions,Project Count,Is Benefit Disadvantaged Communities,Estimated Number Of Trees To Be Planted,Estimated Acres Restored,Estimated Acres Treated
0,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clean Vehicle Rebate Project,NaN,CVRP promotes clean vehicle adoption in Califo...,"Zero-Emission Vehicles, Equipment, and Infrast...",Light-Duty Vehicles,Consumer Incentives,6.001400e+09,50.0,7.0,0.0,0.0,0.0,0.0
1,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clean Vehicle Rebate Project,NaN,CVRP promotes clean vehicle adoption in Califo...,"Zero-Emission Vehicles, Equipment, and Infrast...",Light-Duty Vehicles,Consumer Incentives,6.001400e+09,41.0,7.0,0.0,0.0,0.0,0.0
2,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clean Vehicle Rebate Project,NaN,CVRP promotes clean vehicle adoption in Califo...,"Zero-Emission Vehicles, Equipment, and Infrast...",Light-Duty Vehicles,Consumer Incentives,6.001400e+09,97.0,14.0,0.0,0.0,0.0,0.0
3,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clean Vehicle Rebate Project,NaN,CVRP promotes clean vehicle adoption in Califo...,"Zero-Emission Vehicles, Equipment, and Infrast...",Light-Duty Vehicles,Consumer Incentives,6.001400e+09,68.0,10.0,1.0,0.0,0.0,0.0
4,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clean Vehicle Rebate Project,NaN,CVRP promotes clean vehicle adoption in Califo...,"Zero-Emission Vehicles, Equipment, and Infrast...",Light-Duty Vehicles,Consumer Incentives,6.001400e+09,29.0,4.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133693,Climate Resilience Planning,Works with local governments on adaptation pla...,Climate Change Adaptation and Coastal Resilien...,State Operations,BCDC's Regulatory staff conducts comprehensive...,Climate Adaptation and Resilience,Wetland Restoration and Management,Permitting,NaN,0.0,1.0,0.0,0.0,0.0,0.0
133694,Climate Resilience Planning,Works with local governments on adaptation pla...,Climate Change Adaptation and Coastal Resilien...,State Operations,BCDC improves expertise on climate change issu...,Climate Adaptation and Resilience,Sea Level Rise,Capacity-Building,NaN,0.0,1.0,0.0,0.0,0.0,0.0
133695,Climate Resilience Planning,Works with local governments on adaptation pla...,Climate Change Adaptation and Coastal Resilien...,State Operations,BCDC's Regulatory staff conducts comprehensive...,Climate Adaptation and Resilience,Wetland Restoration and Management,Permitting,NaN,0.0,1.0,0.0,0.0,0.0,0.0
133696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Rollover climate risk keywords subset from v1 crosswalk

In [ ]:
climate_risk_dict = {
    'wildfire mitigation': ['wildfire', 'Wildfire', 'burn', 'controlled burns', 'firefighting', 'forest', 'reforest', 'reforestation', 'fire', 'vegetation management', 'roadside brushing', 'fuel break'],
    'sea level rise mitigation': ['sea level rise', 'slr', 'erosion', 'seawall', 'riparian', 'seawalls', 'shoreline', 'wetland', 'mangrove'],
    'extreme heat mitigation': ['heat', 'extreme heat', 'shade', 'shading', 'cooling center', 'cooling centers', 'heat-resistant', 'heat resistant', 'heat reducing', 'heat-reducing'],
    'drought mitigation': ['drought', 'dry', 'irrigation', 'soil moisture', 'rainwater harvest', 'rainwater harvesting', 'water storage', 'water allocation', 'water management'],
    'inland flooding mitigation': ['flood', 'flooding', 'inland flood', 'inland flooding', 'floodplain', 'floodproof', 'floodproofing', 'elevated flood', 'flood barrier', 'flood barriers', 'drainage'],
    'greenhouse gas mitigation': ['ghg', 'GHG', 'greenhouse gas', 'emission', 'emissions', 'carbon sequestration', 'electrification', 'carbon capture', 'solar power', 'wind energy', 'hydroelectricity', 'geothermal energy', 'biomass energy', 'Energy-efficiency']
}


## How many unique entries for each column? Will help decide which column to start with

In [30]:
import pandas as pd

def count_entries(dataframe):
    total_entries = dataframe.count()
    unique_entries = dataframe.nunique()
    return total_entries, unique_entries

# Example usage:
total_entries, unique_entries = count_entries(data_of_interest)
print("Total entries per column:")
print(total_entries)
print("\nUnique entries per column:")
print(unique_entries)


Total entries per column:
Program Name                   133696
Program Description            133694
Sub Program Name               133696
Project Type                    19903
Project Description            133696
SECTOR                         133697
CATEGORY                       133697
ACTION                         133697
Census Tract                   114823
Total Project GHGReductions    133696
Project Count                  133696
dtype: int64

Unique entries per column:
Program Name                     39
Program Description              19
Sub Program Name                 76
Project Type                    963
Project Description            5235
SECTOR                           25
CATEGORY                         77
ACTION                           95
Census Tract                   7978
Total Project GHGReductions    2604
Project Count                   243
dtype: int64


## Deciding to loop through 'Project Description' first as it has a large number of variation to capture many datasets, and makes most practical scense for filtering climate keywords